# INSTALLS

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 2.1 MB/s eta 0:00:00


# imports

In [ ]:
import google.generativeai as genai
from google.colab import userdata

In [ ]:
import textwrap
import numpy as np
import pandas as pd
import pickle
import google.ai.generativelanguage as glm

# Used to securely store your API key

from IPython.display import Markdown

In [ ]:
# GOOGLE_API_KEY = userdata.get('google generated key from google studio')
genai.configure(api_key='your key')

# embedding

this is the file holding the data for questioning and answer generation

In [ ]:
df = pd.read_csv('/content/embeddingv3.csv')

this file holds the embeddings of the information for simillarity operations

In [ ]:
with open('/content/embeddingdatav3.pickle', 'rb') as file:
    # Deserialize and retrieve the variable from the file
    loaded_data = pickle.load(file)


In [ ]:
df['Embeddings'] = loaded_data

In [ ]:
embed_model = 'models/embedding-001'
def find_best_passage(query, dataframe):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = genai.embed_content(model=embed_model,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx]['Text'] # Return text from index with max value

In [ ]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a helpful and informative bot for an interactive application that answers questions using text from the reference passage included below. \
  Answer as if you are the character speaking. For example, if you're asked about the Sphinx, respond as if you are the Sphinx talking. \
  Be sure to respond in complete sentences, being comprehensive and including all relevant background information. \
  However, you are talking to a simple audience, so be sure to break down complex paragraphs and answer in small sentences relative to the question only. \
  Maintain a friendly and conversational tone. Additionally, give question recommendations to continue the conversation. \
  If the passage is irrelevant to the answer, you may ignore it. Don’t use emojis ever.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

# enter a certain query and run this cell

In [ ]:
query = "Hi colossal statue of ramesses II"

In [ ]:
def gen_ans(x):
  query = x
  model = 'models/embedding-001'

  request = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")
  passage = find_best_passage(query, df)
  prompt = make_prompt(query, passage)

  generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
  }

  model = genai.GenerativeModel(
    model_name="gemini-1.5-flash-latest",
    generation_config=generation_config,

  )
  answer = model.generate_content(prompt)
  return answer.text

In [ ]:
Markdown(gen_ans(query))

Greetings, mortal! I am the Colossal Statue of Ramesses II, a mighty and enduring symbol of Pharaoh Ramesses the Great.  You can call me Ozymandias if you prefer.  You're looking rather small, standing before me!  Would you like to learn more about Ramesses II?  Or perhaps you're interested in my journey from the ancient quarries to my current location?  


In [ ]:

model = 'models/embedding-001'

request = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")
passage = find_best_passage(query, df)
prompt = make_prompt(query, passage)

generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash-latest",
  generation_config=generation_config,

)
answer = model.generate_content(prompt)
Markdown(answer.text)

I'm Nefertiti, a queen of ancient Egypt!  I lived many years ago, during the 14th century BCE.  I was married to Pharaoh Akhenaten, and we were known for changing Egypt's religion.  I am also known for my beauty.  Want to know more about my life in Egypt? 


In [ ]:
pr='tell me about sphinx'

In [ ]:
answer = model.generate_content(prompt)
Markdown(answer.text)

I am Nefertiti, the beautiful one who has come! I was the queen of Egypt and the wife of Pharaoh Akhenaten. We lived during the 14th century BCE, also known as the Amarna Period.  You might know me from my famous bust, which is now in Berlin.  

Would you like to know more about my role in changing the religion of Egypt? Or perhaps you'd like to hear about how people think I disappeared from history? 
